In [36]:
import pandas as pd
import mysql.connector as msql
from bokeh.io import show, output_notebook
from bokeh.models import ColumnDataSource, FactorRange
from bokeh.palettes import viridis
from bokeh.plotting import figure
from bokeh.transform import factor_cmap

In [24]:
#list groupings for Nova, BB, Neptune
#for future part numbers >>> nova_260_5k = [] nova_260_4k = []
d = {
    'nova_120_5k' : ['BB100-50k277DIA-HK', 'BB136S12850KHK14L', 'BB136S12850KHK14LMS', 'BB136S12850KHK18L',
            'BB136S14950KHK14L', 'BB136S14950KHK18L','BB150-50k277DIA18L-HK','BB150-50k277DIA-HK',
            'BB150-50k277DIA-MD', 'BB150-50k480DIA-HK', 'NEP120W48V50KHK', 'NEP120WUNV50KHK',
            'NEP120WUNV50KHK-MS', 'NEP120WUNV50KP3', 'NOV120WUNV50HK'],
    'nova_120_4k' : ['BB136S12840KHK14L', 'NEP120W48V40KHK', 'NEP120WUNV40KHK','NEP120WUNV40KHK-80',
               'NEP120WUNV40KHKC-W', 'NEP120WUNV40KHK-G', 'NEP120WUNV40KP3'],
    'nova_150_5k' : ['NEP150WUNV50KHK', 'NEP150WUNV50KP3', 'NEP150W4850KHK', 'NEP150WUNV50KHKW15', 
               'NAU150WUNV50KHKP', 'NAU150W48V50KHK', 'NAU150WUNV50KHKP-B'],
    'nova_150_4k' : ['NEP150WUNV40KHK', 'NEP150WUNV40KHK-80', 'NEP150W2840K18LHK', 'NAU150WUNV40KHKPC',
               'NAU150WUNV40KHKPCMS', 'NAU150WUNV40KHKP-B','NAU150WUNV40KHKP', 'NAU150WUNV40KHKPMS'],
    'nova_200_5k' : ['NAU200WUNV50KHKP','NAU200WUNV50KHK', 'NAU200W48V50KHKP', 'NAU200WUNV50KHKP-B'],
    'nova_200_4k' : ['NAU200WUNV40KHK', 'NAU200WUNV40KHKPDIM', 'NAU200WUNV40KHKP', 'NAU201E200W2840KHK']
}
#create db connection
cnx = msql.connect(user='gone', password='fishing', port = '3305', host='192.168.88.88', database='fintronx_llc_live')

#store sql query
nep_nau_nov_sold = """
            select soi.productNum as Product_Number, date_format(so.dateCreated, '%Y') as Year, cast(quarter(so.dateCreated) as char) as Quarter, soi.qtyOrdered as Qty_Ordered
            from fintronx_llc_live.soitem soi
            join fintronx_llc_live.so so
            on soi.soId = so.id
            where ((soi.productNum like 'BB%' or soi.productNum like 'NEP%' or soi.productNum like 'NAU%' or soi.productNum like 'NOV%') and (so.customerPO not like 'FX%' and so.customerPO not like '%DEMO%'))
            and soi.typeId = 10;
            """
#run/read the sql file
sold = pd.read_sql(nep_nau_nov_sold, cnx)

#replace original p/n's with grouped labels
for k, v in d.items():
    for each in v:
        sold.replace(each, k, inplace = True)

#add 'Qtr' label to Quarter column
sold['Quarter'] = 'Qtr' + sold['Quarter']

#group by part number, year, quarter
soldg = sold.groupby(['Product_Number', 'Year', 'Quarter']).sum()

soldg


Qty_Ordered
Product_Number Year Quarter             
NEPHK          2016 Qtr3            10.0
               2018 Qtr1             7.0
                    Qtr3            50.0
nova_120_4k    2016 Qtr2             1.0
                    Qtr4            14.0
               2017 Qtr3           107.0
                    Qtr4           653.0
               2018 Qtr1            10.0
                    Qtr2            55.0
                    Qtr3           457.0
                    Qtr4            59.0
               2019 Qtr1            28.0
nova_120_5k    2015 Qtr1           317.0
                    Qtr2          1006.0
                    Qtr3           759.0
                    Qtr4          1107.0
               2016 Qtr1           698.0
                    Qtr2           874.0
                    Qtr3           752.0
                    Qtr4           525.0
               2017 Qtr1           423.0
                    Qtr2           746.0
                    Qtr3           537.0
                    Qtr4           481.0
               2018 Qtr1           687.0
                    Qtr2           239.0
                    Qtr3           237.0
                    Qtr4           695.0
               2019 Qtr1           325.0
nova_150_4k    2016 Qtr2            39.0
...                                  ...
               2018 Qtr3           266.0
                    Qtr4             4.0
               2019 Qtr1            10.0
nova_150_5k    2016 Qtr3            99.0
                    Qtr4           158.0
               2017 Qtr1           105.0
                    Qtr2            71.0
                    Qtr3            27.0
                    Qtr4           204.0
               2018 Qtr1           100.0
                    Qtr2            88.0
                    Qtr3            45.0
                    Qtr4            46.0
               2019 Qtr1            32.0
nova_200_4k    2016 Qtr2           133.0
                    Qtr3            12.0
               2017 Qtr1             4.0
               2018 Qtr1            16.0
                    Qtr2             5.0
                    Qtr3             5.0
nova_200_5k    2016 Qtr4             6.0
               2017 Qtr1            55.0
                    Qtr2             4.0
                    Qtr3             3.0
                    Qtr4            25.0
               2018 Qtr1            11.0
                    Qtr2            30.0
                    Qtr3            45.0
                    Qtr4            17.0
               2019 Qtr1             1.0

[67 rows x 1 columns]

In [27]:
output_notebook()
idx = soldg.index.values

Loading BokehJS ...

In [54]:
#load dataframe into bokeh data source
source = ColumnDataSource(soldg)

colors = viridis(4)

#cmap with factors based on indices
idx_cmap = factor_cmap('Product_Number_Year_Quarter',  palette = colors, factors = sold.Quarter.unique(), start = 1, end = 2)

#create figure, pass multi level index to FactorRange and unpack into x_range
p = figure(plot_width = 2000, plot_height = 500, title = 'IDK', x_range = FactorRange(*idx))

#create a vertical bar chart with indices as x axis and qty column as y axis
p.vbar(x = 'Product_Number_Year_Quarter', top = 'Qty_Ordered', width = .5,  source = source)


GlyphRenderer(id='2627', ...)

In [55]:
show(p)